<font color = magenta> Lixandru Andreaa-Bianca Grupa10LF382
<br> 
<font color = magenta> Pepene Adina-FLorentina Grupa10LF382

# Laborator 9

## Modele de regresie

Folositi urmatoarele seturi de date:
1. [CPU Computer Hardware](https://archive.ics.uci.edu/ml/datasets/Computer+Hardware); excludeti din dataset coloanele: vendor name, model name, estimated relative performance; se va estima coloana "published relative performance".
1. [Boston Housing](http://archive.ics.uci.edu/ml/machine-learning-databases/housing/)
1. [Wisconsin Breast Cancer](http://www.dcc.fc.up.pt/~ltorgo/Regression/DataSets.html); cautati in panelul din stanga Wisconsin Breast Cancer si urmati pasii din "My personal Notes"
1. [Communities and Crime](http://archive.ics.uci.edu/ml/datasets/communities+and+crime); stergeti primele 5 dimensiuni si trasaturile cu missing values.

Pentru fiecare set de date aplicati minim 5 modele de regresie din scikit learn. Pentru fiecare raportati: mean absolute error, mean squared error, median absolute error - a se vedea [sklearn.metrics](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) - folosind 5 fold cross validation. Valorile hiperparametrilor trebuie cautate cu grid search (cv=3)  si random search (n_iter dat de voi). Metrica folosita pentru cautarea hiperparametrilor va fi mean squared error. Raportati mediile rezultatelor atat pentru fold-urile de antrenare, cat si pentru cele de testare; indicatie: puteti folosi metoda `cross_validate` cu parametrul `return_train_score=True`, iar ca model un obiect de tip `GridSearchCV` sau `RandomizedSearchCV`.

Rezultatele vor fi trecute intr-un dataframe. Intr-o stare intermediara, valorile vor fi calculate cu semnul minus: din motive de implementare, biblioteca sklearn transforma scorurile in numere negative; a se vedea imaginea de mai jos:

![intermediate report](./images/cpu_intermediate_blurred.png)


Valorile vor fi aduse la interval pozitiv, apoi vor fi marcate cele maxime si minime; orientativ, se poate folosi imaginea de mai jos, reprezentand dataframe afisat in notebook; puteti folosi alte variante de styling pe dataframe precum la https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#.  

Se va crea un raport final in format HTML sau PDF - fisier(e) separat(e). Raportul trebuie sa contina minimal: numele setului de date si obiectul dataframe; preferabil sa se pastreze marcajul de culori realizat in notebook.

![report](./images/cpu_results_blurred.png)

Notare:
1. Se acorda 20 de puncte din oficiu.
1. Optimizare si cuantificare de performanta a modelelor: 3 puncte pentru fiecare combinatie set de date + model = 60 de puncte
1. Documentare modele: numar modele * 2 puncte = 10 puncte. Documentati in jupyter notebook fiecare din modelele folosite, in limba romana. Puteti face o sectiune separata cu documentarea algoritmilor. Fiecare model trebuie sa aiba o descriere de minim 20 de randuri, minim o imagine asociata si minim 2 referinte bibliografice.
1. 10 puncte: export in format HTML sau PDF.



*Notare:* rezolvarea va fi incarcata pe platforma de elearning in saptamana 11-15 mai.

# Citirea si separarea datelor 

In [76]:
import pandas as pd 
import numpy as np
from typing import List

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from statistics import mean 

In [77]:
dataframe_cpu_names = ['vendor_names', 'model', 'myct', 'mmin', 'mmax', 'cach', 'chmin', 'chmax', 'prp', 'erp']

dataframe_cpu = pd.read_csv('data\machine\machine.data', names=dataframe_cpu_names, header=None)

dataframe_cpu = dataframe_cpu.drop(columns=['vendor_names', 'model', 'erp'])

dataframe_cpu.head()

,myct,mmin,mmax,cach,chmin,chmax,prp
0,125,256,6000,256,16,128,198
1,29,8000,32000,32,8,32,269
2,29,8000,32000,32,8,32,220
3,29,8000,32000,32,8,32,172
4,29,8000,16000,32,8,16,132


In [78]:
dataframe_cpu_x = dataframe_cpu.values[:, :-1]
dataframe_cpu_y = dataframe_cpu.values[:, -1]

In [79]:
dataframe_housing_names = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis',
                           'rad', 'tax', 'ptratio', 'b', 'lstat', 'medv']

dataframe_housing = pd.read_csv('data\housing\housing.data', names=dataframe_housing_names, header = None, sep=r"\s+")

dataframe_housing.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [80]:
dataframe_housing_x = dataframe_housing.values[:, :-1]
dataframe_housing_y = dataframe_housing.values[:, -1]

# Cautarea hiperparametrilor 

In [81]:
def search_grid(model, parameters:dict , x_train:np.ndarray, y_train:np.ndarray, x_test:np.ndarray, y_test:np.ndarray):
    
    '''This function takes the parameters and search for best parameters.
    
    Parameters:
        model - model from sklearn 
        parameters - all the alternatives to the hyperparameters
        x_train, y_train - input and output values for model train
        x_test, y_test - input and output values for model test
    
    It returns the errors.
    '''
    
    grid_search = GridSearchCV(estimator=model, param_grid=parameters, scoring='neg_mean_squared_error', cv=3,  return_train_score=True)
    grid_search.fit(x_train, y_train)
    
    return errors(grid_search, x_train, y_train, x_test, y_test)

In [82]:
def search_random(model, parameters:dict , x_train:np.ndarray, y_train:np.ndarray, x_test:np.ndarray, y_test:np.ndarray):
    
    '''This function takes the parameters and search for best parameters.
    
    Parameters:
        model - model from sklearn 
        parameters - all the alternatives to the hyperparameters
        x_train, y_train - input and output values for model train
        x_test, y_test - input and output values for model test
    
    It returns the errors.
    '''
    rand_search = RandomizedSearchCV(estimator=model, param_distributions=parameters, scoring='neg_mean_squared_error', cv=3,  return_train_score=True)
    rand_search.fit(x_train, y_train)
    
    return errors(rand_search, x_train, y_train, x_test, y_test)

# Calcularea erorilor 

In [83]:
def errors(model, x_train:np.ndarray, y_train:np.ndarray, 
           x_test:np.ndarray, y_test:np.ndarray)-> List[float]:
    
    '''Errors function calculates mean absolute error, 
    mean squared error and median absolute error for model.
    
    Parameters:
        model - model from sklearn 
        x_train, y_train - input and output values for model train
        x_test, y_test - input and output values for model test
        
    Returns a list with mean of errors:
    neg_mean_absolute_error for train and test data
    neg_mean_squared_error for train and test data
    neg_median_absolute_error for train and test data
    '''
    
    scores_neg_mean_abs_err_train = (-1)*cross_val_score(model, x_train, y_train, cv = 5, scoring = 'neg_mean_absolute_error').mean()
    scores_neg_mean_abs_err_test = (-1)*cross_val_score(model, x_test, y_test, cv = 5, scoring = 'neg_mean_absolute_error').mean()

    scores_neg_mean_sq_err_train = (-1)*cross_val_score(model, x_train, y_train, cv = 5, scoring = 'neg_mean_squared_error').mean()
    scores_neg_mean_sq_err_test = (-1)*cross_val_score(model, x_test, y_test, cv = 5, scoring = 'neg_mean_squared_error').mean()
    
    scores_neg_med_abs_err_train = (-1)*cross_val_score(model, x_train, y_train, cv = 5, scoring = 'neg_median_absolute_error').mean()
    scores_neg_med_abs_err_test = (-1)*cross_val_score(model, x_test, y_test, cv = 5, scoring = 'neg_median_absolute_error').mean()
    
    result = [scores_neg_mean_abs_err_train, scores_neg_mean_abs_err_test,
              scores_neg_mean_sq_err_train, scores_neg_mean_sq_err_test,
              scores_neg_med_abs_err_train, scores_neg_med_abs_err_test]
    
    return result

# K-Nearest Neighbor Regression(KNN)

In [84]:
def KNN(model, parameters:dict, x_train:np.ndarray, y_train:np.ndarray, x_test:np.ndarray, y_test:np.ndarray):
    
    '''KNN function get params and do regression based on k-nearest
    neighbors with best parameters generated by grid search and random search.
    It generates the error values.
    
    Parameters:
        model - model from sklearn 
        parameters - all the alternatives to the hyperparameters
        x_train, y_train - input and output values for model train
        x_test, y_test - input and output values for model test 
    '''
    
    print ("KNN", search_grid(model, parameters, x_train, y_train, x_test, y_test))
    print ()
    print ("KNN", search_random(model, parameters, x_train, y_train, x_test, y_test))    

# Decision Tree Regression

In [85]:
def decision_tree(model, parameters:dict, x_train:np.ndarray, y_train:np.ndarray, x_test:np.ndarray, y_test:np.ndarray):
    
    '''decision_tree function get params and do regression based on k-nearest
    neighbors with best parameters generated by grid search and random search.
    It generates the error values.
    
    Parameters:
        model - model from sklearn 
        parameters - all the alternatives to the hyperparameters
        x_train, y_train - input and output values for model train
        x_test, y_test - input and output values for model test 
    '''
    
    print ("Decision tree", search_grid(model, parameters, x_train, y_train, x_test, y_test))
    print ()
    print ("Decision tree", search_random(model, parameters, x_train, y_train, x_test, y_test))    

# Aplicarea modelelor pentru generarea erorilor 

In [86]:
def start(x, y):
    
    model_knn = KNeighborsRegressor()
    model_decision_tree = DecisionTreeRegressor()

    parameters_knn = {'n_neighbors': [3, 4, 5, 6, 8, 9, 10, 11], 'p': [2, 3, 4, 5]}
    parameters_decision_tree = {'criterion': ['mse', 'friedman_mse'], 'max_depth': [3, 4, 5, 6, 7, 8]}
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1/3)

    decision_tree(model_decision_tree, parameters_decision_tree, x_train, y_train, x_test, y_test)
    KNN(model_knn, parameters_knn, x_train, y_train, x_test, y_test)

In [87]:
start(dataframe_cpu_x, dataframe_cpu_y)

Decision tree [34.43845557216772, 50.58154761904761, 7472.485128888735, 14093.781254676873, 12.749682539682539, 26.304960317460313]

Decision tree [39.00410090868706, 51.10272370486656, 8154.224963501588, 13393.481705296188, 15.168626110731372, 22.969404761904762]
KNN [36.977513227513235, 45.64095238095238, 5044.218959435626, 13473.80428571429, 13.366666666666665, 20.21333333333333]

KNN [38.09944885361552, 45.66666666666667, 5378.160319297472, 13840.437797619044, 13.366666666666665, 20.93]


In [88]:
start(dataframe_housing_x, dataframe_housing_y)

Decision tree [3.1843289438696987, 3.680714546576721, 25.54356646371312, 28.416172747449423, 2.1532699841412706, 2.4400141898864787]

Decision tree [3.4805413771030613, 3.7210833303428417, 24.554647711055946, 27.791017390273915, 2.2651588730301606, 2.4801694389426263]
KNN [4.6494776119402985, 4.901451574569221, 45.66102318798166, 48.88175789859378, 3.1453333333333346, 3.5760000000000005]

KNN [4.790644205443372, 5.010717468805704, 45.410689652933584, 50.03384633034895, 3.1660000000000013, 3.772277777777778]


# Documentatie

## 1. K-Nearest Neighbors Regression(KNN)

K-Nearest Neighbors Regression este un algoritm simplu care stochează toate cazurile posibile și prezice o valoare pe baza unei măsuri de similaritate (de exemplu, funcții de distanță). KNN a fost utilizat în estimarea statistică și recunoașterea modelului de la începutul anilor 1970 ca tehnică non-parametrică. Regresia KNN utilizează aceleași funcții de distanță ca și clasificarea KNN. K reprezinta numarul de vecini care este cautat pentru a-l determina pe cel mai potrivit.
<br><br>Când __K = 1__ algoritmul este cunoscut ca nearest neighbor. Acesta este cel mai simplu caz. Sa presupunem că P1 este un punct pentru care trebuie sa ii prezicem eticheta. Mai întâi, găsim cel mai apropiat punct de P1 și atribuim eticheta acestuia lui P1.

<img src="resources/image1.png" width="400">

Un alt exemplu pentru __K = 3__ este:

<img src="resources/image2.png" width="400">

Urmarind plot-urile de mai jos putem observa ca folosind doar un singur vecin, fiecare punct din setul de antrenare are o influență evidentă asupra predicțiilor, iar valorile prezise trec prin toate punctele. Aceasta duce la o predicție foarte nesigură. Considerând mai mulți vecini se ajunge la predicții mai ușoare, dar acestea nu se potrivesc și cu datele de instruire.

<img src="resources/image3.png" width="1000">

Regresia K-Nearest Neighbors poate fi utilizată în cazurile în care etichetele de date sunt continue, mai degrabă decât variabile discrete. Eticheta atribuită unui punct de interogare este calculată pe baza mediilor etichetelor celor mai apropiați vecini.
În faza de clasificare, k este o constantă definită de utilizator, iar un vector nemarcat (o interogare sau un punct de testare) este clasificat prin atribuirea etichetei care este cea mai frecventă dintre eșantioanele de pregătire k cele mai apropiate de punctul de interogare.

__Metode de calcul al distantei dintre doua puncte__

Algoritmul KNN este un algoritm de invatare supervizata, care foloseste distante pentru a gasi similitudini si diferente.

__1. Distanta Euclidiana:__ este cea mai frecventă și măsoară distanța liniară dintre două probe
__2. Distanta Manhattan:__ măsoară timpul de călătorie punct-la-punct și este utilizata în mod obișnuit pentru predictorii binari

<img src="resources/image4.png" width="250">

__3. Distanta Hamming__

<img src="resources/image5.png" width="150">


Distanta Euclidiana si distanta Manhattan sunt folosite pentru variabile continue, iar distanta Hamming este folosita pentru variabile categoriale.

Cum aflam care este cel mai potrivit K?
Daca folosim un K foarte mic vom face overfitting pe setul de antrenare si nu vom avea rezultate deloc bune pentru setul de testare. Daca folosim un K foarte mare nu vom obtine rezultate bune nici pe setul de antrenare, nici pe setul de intrare. Ce facem? Solutia este curba 'elbow curve'. Un K bun poate fi selectat prin diferite tehnici euristice (ex: optimizarea hiperparametrului). 

<img src="resources/image6.png" width="550">

Un dezavantaj al clasificării de bază a „votului majoritar” apare atunci când exemple dintr-o clasă mai frecventă tind să domine predicția noului exemplu, deoarece acestea tind să fie comune printre cei mai apropiați k din cauza numărului mare. O modalitate de a depăși această problemă este ponderea clasificării, ținând cont de distanța de la punctul de testare la fiecare dintre cei mai apropiați k vecini ai săi. Clasa (sau valoarea, în problemele de regresie) din fiecare dintre cele mai apropiate k puncte este înmulțită cu o greutate proporțională cu inversul distanței de la acel punct la punctul de testare. 

Precizia algoritmului KNN poate fi sever degradată prin prezența unor caracteristici zgomotoase sau irelevante sau dacă scările de caracteristici nu sunt în concordanță cu importanța lor. 

## Decision Tree Regression

### Bibliografie:

1. https://www.saedsayad.com/k_nearest_neighbors_reg.htm
2. https://medium.com/analytics-vidhya/k-neighbors-regression-analysis-in-python-61532d56d8e4
3. https://scikit-learn.org/stable/modules/neighbors.html#regression
4. https://www.analyticsvidhya.com/blog/2018/08/k-nearest-neighbor-introduction-regression-python/
5. https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm